In [1]:
import numpy as np
from collections import Counter

def generate_kmers(sequence, k):
    """Extract all k-mers from a sequence."""
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

def build_kmer_dict(sequences, k):
    """Create a dictionary mapping each unique k-mer to an index."""
    kmer_set = set()
    for seq in sequences:
        kmer_set.update(generate_kmers(seq, k))
    return {kmer: idx for idx, kmer in enumerate(sorted(kmer_set))}

def compute_kmer_feature_matrix(sequences, k, kmer_dict):
    """Convert sequences into k-mer frequency vectors."""
    num_samples = len(sequences)
    num_kmers = len(kmer_dict)
    feature_matrix = np.zeros((num_samples, num_kmers))

    for i, seq in enumerate(sequences):
        kmer_counts = Counter(generate_kmers(seq, k))
        for kmer, count in kmer_counts.items():
            if kmer in kmer_dict:
                feature_matrix[i, kmer_dict[kmer]] = count
    return feature_matrix

def spectrum_kernel_matrix(sequences, k):
    """Compute the Spectrum Kernel matrix."""
    kmer_dict = build_kmer_dict(sequences, k)
    feature_matrix = compute_kmer_feature_matrix(sequences, k, kmer_dict)
    return feature_matrix @ feature_matrix.T  # Compute the dot product

# Example usage
sequences = ["AGCT", "GCTA", "CTAG"]
k = 2
K = spectrum_kernel_matrix(sequences, k)
print(K)


[[3. 2. 2.]
 [2. 3. 2.]
 [2. 2. 3.]]


In [ ]:
import numpy as np
import cvxopt

def train_svm(K, y, C=1.0):
    """Train an SVM using the precomputed kernel matrix K."""
    n = len(y)
    y = y.astype(float).reshape(-1, 1)  # Ensure y is a column vector

    # Construct the quadratic programming matrices
    P = cvxopt.matrix(np.outer(y, y) * K)  # P_ij = y_i * y_j * K_ij
    q = cvxopt.matrix(-np.ones(n))        # q_i = -1
    G = cvxopt.matrix(np.vstack((-np.eye(n), np.eye(n))))  # Constraints 0 <= α <= C
    h = cvxopt.matrix(np.hstack((np.zeros(n), C * np.ones(n))))
    A = cvxopt.matrix(y.T)  # Equality constraint sum(α_i * y_i) = 0
    b = cvxopt.matrix(0.0)

    # Solve the quadratic program
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(solution['x'])

    # Support vectors: α > 0
    sv_indices = alphas > 1e-5
    support_vectors = np.where(sv_indices)[0]
    alphas = alphas[sv_indices]
    support_y = y[sv_indices].flatten()

    # Compute the bias term (intercept)
    bias = np.mean(support_y - np.sum(alphas * support_y * K[support_vectors][:, support_vectors], axis=1))

    return alphas, support_vectors, bias

def predict_svm(K_test, alphas, support_vectors, support_y, bias):
    """Make predictions using the trained SVM."""
    return np.sign(np.sum(alphas * support_y * K_test[:, support_vectors], axis=1) + bias)

# Example usage
sequences_train = ["AGCT", "GCTA", "CTAG", "TAGC"]
sequences_test = ["AGCT", "CTAG"]

y_train = np.array([1, -1, 1, -1])  # Example labels

k = 2
K_train = spectrum_kernel_matrix(sequences_train, k)
alphas, support_vectors, bias = train_svm(K_train, y_train)

K_test = spectrum_kernel_matrix(sequences_test + sequences_train, k)[:len(sequences_test), :]
predictions = predict_svm(K_test, alphas, support_vectors, y_train[support_vectors], bias)

print("Predictions:", predictions)


Next Steps

    Tune the C parameter.
    Try different values of k (k-mer size).
    Optimize performance (e.g., using sparse representations for large sequences).